<a href="https://colab.research.google.com/github/kaysriram/MLZoomCamp2025/blob/main/DataTalksHW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Importing Dataset

In [3]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-16 07:59:45--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.02s   

2025-10-16 07:59:45 (4.56 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [4]:
df=pd.read_csv('course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [5]:
df.isnull().sum()

,0
lead_source,128
industry,134
number_of_courses_viewed,0
annual_income,181
employment_status,100
location,63
interaction_count,0
lead_score,0
converted,0


In [16]:
df[df.select_dtypes(include='object').columns] = df[df.select_dtypes(include='object').columns].fillna('NA')
df[df.select_dtypes(exclude='object').columns] = df[df.select_dtypes(exclude='object').columns].fillna(0.0)

In [17]:
df.isnull().sum()

,0
lead_source,0
industry,0
number_of_courses_viewed,0
annual_income,0
employment_status,0
location,0
interaction_count,0
lead_score,0
converted,0


Q1

In [19]:
df['industry'].value_counts().idxmax()   #gives the index of maximum value(mode)

'retail'

Q2

In [30]:
df_num=df[df.select_dtypes(exclude='object').columns]
df_num.corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [40]:
corr_mat=df_num.corr().unstack().sort_values(ascending=False)
corr_notself=corr_mat[corr_mat<1]

In [42]:
corr_notself      #interaction_count and lead_score,number_of_courses_viewed and lead_score,number_of_courses_viewed and interaction_count,annual_income and interaction_count


converted                 number_of_courses_viewed    0.435914
number_of_courses_viewed  converted                   0.435914
interaction_count         converted                   0.374573
converted                 interaction_count           0.374573
lead_score                converted                   0.193673
converted                 lead_score                  0.193673
                          annual_income               0.053131
annual_income             converted                   0.053131
interaction_count         annual_income               0.027036
annual_income             interaction_count           0.027036
                          lead_score                  0.015610
lead_score                annual_income               0.015610
interaction_count         lead_score                  0.009888
lead_score                interaction_count           0.009888
annual_income             number_of_courses_viewed    0.009770
number_of_courses_viewed  annual_income               0.009770
lead_score                number_of_courses_viewed   -0.004879
number_of_courses_viewed  lead_score                 -0.004879
                          interaction_count          -0.023565
interaction_count         number_of_courses_viewed   -0.023565
dtype: float64

On comparing the given pairs, annual income and interaction count has highest correlation

Q3

In [64]:
from sklearn.model_selection import train_test_split
X=df.drop(columns=['converted'])
y=df['converted']
X_train,X_temp,y_train,y_temp=train_test_split(X,y,test_size=0.4,random_state=42)      #60 40 split
X_val,X_test,y_val,y_test=train_test_split(X_temp,y_temp,test_size=0.5,random_state=42) # 40 split into 20 20 each

In [65]:
X_train_cat=X_train[X_train.select_dtypes(include='object').columns]
X_train_num=X_train[X_train.select_dtypes(exclude='object').columns]

In [66]:
from sklearn.metrics import mutual_info_score
def calc_mi(series):
  return round(mutual_info_score(series,y_train),2)

mi_scores = X_train_cat.apply(calc_mi).sort_values(ascending=False)
display(mi_scores)

,0
lead_source,0.03
industry,0.02
employment_status,0.02
location,0.00


AttributeError: 'NoneType' object has no attribute 'idxmax'

In [67]:
mi_scores.idxmax() #max mutual info with converted column

'lead_source'

Q4

Instead of doing it the way in the tutorial. We'll implement OneHot Encoding using sklearn

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score



In [68]:
cat_cols=X_train.select_dtypes(include='object').columns
num_cols=X_train.select_dtypes(exclude='object').columns

In [70]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ]) #preprocessed data


In [71]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [74]:
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])
clf.fit(X_train,y_train)
y_val_pred = clf.predict(X_val)
val_acc=accuracy_score(y_val,y_val_pred)
round(val_acc,2)


0.74

Q5

In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

def accuracy_change():
    diff = []  # local variable

    # Identify categorical and numerical columns
    cat_cols_base = X_train.select_dtypes(include='object').columns
    num_cols_base = X_train.select_dtypes(exclude='object').columns

    # Base preprocessor
    preprocessor_base = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', num_cols_base),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols_base)
        ]
    )

    # Base model and pipeline
    clf_base = Pipeline(steps=[
        ('preprocessor', preprocessor_base),
        ('model', LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42))
    ])

    # Baseline accuracy
    clf_base.fit(X_train, y_train)
    y_val_pred_base = clf_base.predict(X_val)
    val_acc_base = accuracy_score(y_val, y_val_pred_base)

    # Drop one feature at a time
    for col in X_train.columns:
        new_X_train = X_train.drop(columns=[col])
        new_X_val = X_val.drop(columns=[col], errors='ignore')

        # Redefine preprocessor with the new set of columns
        cat_cols_new = new_X_train.select_dtypes(include='object').columns
        num_cols_new = new_X_train.select_dtypes(exclude='object').columns

        preprocessor_new = ColumnTransformer(
            transformers=[
                ('num', 'passthrough', num_cols_new),
                ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols_new)
            ]
        )

        # Create a new pipeline for each feature drop
        clf_new = Pipeline(steps=[
            ('preprocessor', preprocessor_new),
            ('model', LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42))
        ])

        # Train and evaluate
        clf_new.fit(new_X_train, y_train)
        y_val_pred_1feat = clf_new.predict(new_X_val)
        val_acc_1feat = accuracy_score(y_val, y_val_pred_1feat)

        # Record accuracy drop
        diff.append((col, val_acc_base - val_acc_1feat))

    return diff


In [93]:
feature_drops = accuracy_change()
feature_drops.sort(key=lambda x: x[1], reverse=True)
feature_drops[:10]

[('interaction_count', 0.06849315068493145),
 ('number_of_courses_viewed', 0.06506849315068486),
 ('lead_source', 0.013698630136986245),
 ('industry', 0.0),
 ('location', 0.0),
 ('lead_score', 0.0),
 ('employment_status', -0.003424657534246589),
 ('annual_income', -0.113013698630137)]

Industry seems to have no difference. So, this is the answer!

Q6

In [102]:

def reg_lr():
    diff = []

    cat_cols = X_train.select_dtypes(include='object').columns
    num_cols = X_train.select_dtypes(exclude='object').columns

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', num_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
        ]
    )

    # Baseline model (C=1.0)
    base_clf = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42))
    ])
    base_clf.fit(X_train, y_train)
    y_val_pred_base = base_clf.predict(X_val)
    val_acc = accuracy_score(y_val, y_val_pred_base)


    for i in [0.01, 0.1, 1, 10, 100]:
        clf = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', LogisticRegression(solver='liblinear', C=i, max_iter=1000, random_state=42))
        ])
        clf.fit(X_train, y_train)
        y_val_pred = clf.predict(X_val)
        val_acc_reg = accuracy_score(y_val, y_val_pred)

        #change = val_acc - val_acc_reg
        diff.append((i, val_acc_reg))

    return max(diff,key=lambda item:item[1])



In [103]:
reg_lr()

(0.01, 0.7431506849315068)